## Часть 1. Подготовка датасета

В обучении нашей модели мы будем использовать датасет для автопилотируемых машин.

Шаг 1.

Начнем с того, что посмотрим на наш датасет. Внутри куча схожих по названию папок, каждая из которых содержит картинки. 
Но мы можем выделить в этом датасете два вида картинок. 
<br>1) Это обычные цветные картинки.
Например dataA/dataA/CameraRGB/02_00_000.png
<br>2) И есть связанные с ними картинки, разбитые на области с одинаковыми яркостями пикселей.
<br>Например dataA/dataA/CameraSeg/02_00_000.png, в ней все тоже самое, что и в первой, но  она просегментирована.

И еще заметим, что нигде в датасете нет явной информации о классах. Мы должны дать им имена сами.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

Найдем уникальные значения пискселей на картинке, и каждое такое значение будет соответствовать целому классу.

In [ ]:
DATA_ROOT = '/kaggle/input/lyft-udacity-challenge/'

In [ ]:
img = plt.imread(DATA_ROOT + 'dataA/dataA/CameraSeg/02_00_000.png')
plt.imshow(img[..., 0]);

In [ ]:
np.unique(img * 255)

В итоге видим, что у нас 13 классов. Вы можете самостоятельно поотображать семантическую маску 
для каждого класса используя код ниже:

In [ ]:
labels = ['Unlabeled','Building','Fence','Other',
          'Pedestrian', 'Pole', 'Roadline', 'Road',
          'Sidewalk', 'Vegetation', 'Car','Wall',
          'Traffic sign']

In [ ]:
for i in range(13):
    mask = plt.imread(DATA_ROOT + 'dataA/dataA/CameraSeg/02_00_000.png') * 255
    mask = np.where(mask == i, 255, 0)
    mask = mask[:,:,0]
    print(np.unique(mask))
    plt.title(f'class: {i} {labels[i]}')
    plt.imshow(mask)
    plt.show()

Шаг 2.

Теперь приведем наш датасет к удобному виду, для этого сначала разделим все на два списка с rgb картинками и seg.

In [ ]:
cameraRGB = []
cameraSeg = []
for root, dirs, files in os.walk(DATA_ROOT):
    for name in files:
        f = os.path.join(root, name)
        if 'CameraRGB' in f:
            cameraRGB.append(f)
        elif 'CameraSeg' in f:
            cameraSeg.append(f)
        else:
            break

Теперь завернем эти два списка в DataFrame из библиотеки pandas.
В итоге выведем первые пять записей из получившегося датафрейма:

In [ ]:
df = pd.DataFrame({'cameraRGB': cameraRGB, 'cameraSeg': cameraSeg})
# Отсортируем  датафрейм по значениям
df.sort_values(by='cameraRGB',inplace=True)

df.reset_index(drop=True, inplace=True)
# Выведем первые пять значений нашего датафрейма
df.head(5)

Шаг 3. 

Теперь обернем все в кастомный датасет для удобной работы в PyTorch.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torch.nn import functional as F

Создадим класс для кастомного датасета:

In [ ]:
class SelfDrivingDataset(Dataset):
    def __init__(self, data, preprocessing=None):
        # Подаем наш подготовленный датафрейм
        self.data = data
        
        # Разделяем датафрейм на rgb картинки 
        self.image_arr = self.data.iloc[:,0]
        # и на сегментированные картинки
        self.label_arr = self.data.iloc[:,1]
        
        # Количество пар картинка-сегментация
        self.data_len = len(self.data.index)
        
        self.preprocessing = preprocessing

    # переопределяем метод getitem, которым мы достаём объект по индексу
    def __getitem__(self, index):
        # Читаем картинку и сразу же представляем ее в виде numpy-массива 
        # размера 600х800 float-значений, кодируя из bgr в rbg
        img = cv2.cvtColor(cv2.imread(self.image_arr[index]), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 256))

        img = np.asarray(img).astype('float')
        
        if self.preprocessing:
            img = self.preprocessing(img)
            img = torch.as_tensor(img)
        else:
            # Нормализуем изображение в значениях [0, 1]
            img = torch.as_tensor(img) / 255.0
            
        # приводим к необходимому для торча виду: каналы, ширина, высота
        img = img.permute(2,0,1)
        
        # считаем сегментированную картинку через opencv
        masks = []
        mask = cv2.cvtColor(cv2.imread(self.label_arr[index]), cv2.COLOR_BGR2RGB)
#         # через пиллоу появлялись артефакты
#         mask = Image.open(self.label_arr[index])
#         mask = mask.resize((256, 256))
#         mask = np.asarray(mask)
        
        # создаём 13 бинарных масок для 13 классов, чтобы отслеживать
        # как сеть предсказывает каждый класс и считать метрику по классам
        for i in range(13):
            # где маска принимает значение интенсивности пикселя
            # создаём маску: 255 - где объект есть, 0 - где нет
            cls_mask = np.where(mask == i, 255, 0)
            cls_mask = cls_mask.astype('float')
            cls_mask = cv2.resize(cls_mask, (256, 256))
            
            # массив 13-ти масок
            masks.append(cls_mask[:,:,0] / 255)
        
        # переводим в тензор, таб будет 13 каналов для всех масок
        masks = torch.as_tensor(masks, dtype=torch.uint8)    
        
        # возвращаем картинку и предсказание
        return (img.float(), masks)

    # переопределяем метод подсчёта длины (было определено в init)
    def __len__(self):
        return self.data_len

In [ ]:
# инициализируем класс датасета, передавая в него датафрейм
dataset = SelfDrivingDataset(df)

# проверим что всё ок, посмотрев нулевой объект
img, masks = dataset[0]
print(img.shape, masks.shape)
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].imshow(img.permute(1, 2, 0))
ax[1].imshow(masks.permute(1, 2, 0)[..., 10])
plt.show()

В результате картинка 3 канала 256х256 и 13 масок 256х256

Затем разделим наш датасет на тренировочную и тестовую выборки.
И обернем их в наш кастомный класс.

In [ ]:
from sklearn.model_selection import train_test_split

# 70 % в тренировочную выборку, 30 - в тестовую
X_train, X_test = train_test_split(df, test_size=0.3)

# Упорядочиваем индексацию
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

# Оборачиваем каждую выборку в наш кастомный датасет
train_data = SelfDrivingDataset(X_train)
test_data = SelfDrivingDataset(X_test)

И теперь уже обернем то, что получилось в известные нам в pytorch даталоадеры:

In [ ]:
train_data_loader = DataLoader(
    train_data,
    batch_size=8,
    shuffle=True
)
test_data_loader = DataLoader(
    test_data,
    batch_size=4,
    shuffle=False # в тесте лучше ничего не перемешивать
)

In [ ]:
# проверим, что генератор исправен, пройдёмся по 1-й итерации
for img, target in train_data_loader:
    print(img.shape, target.shape)
    print(img[0].min(), img[0].max())
    print(target[0].min(), target[0].max())
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    ax[0].imshow(img[0].permute(1, 2, 0))
    ax[1].imshow(target[0].permute(1, 2, 0)[..., 0])
    break

Получилось 8 - батч-сайз, 3 канала, 256х256 пикселей для картинок
для масок 8 -батч-сайз, 13 каналов, 256х256 пикселей.

Видим что картинка была нормализована - изменяется от 0 до 1. Маска тоже.

Если оставить о 0 до 255, коэф Dice и IoU будут считаться некорректно. Они ожидают что кратинки и маски будут от 0 до 1. Иначе коэффициенты становятся отрицательными.

## Часть 2. Создание модели

**Самописный вариант Unet**

Как мы отметили ранее, в архитектуре присутствует 3х3 двойной сверточный слой следующий за активационной функцией Relu в обеих частях сетки.


Шаг 1.

Создадим функцию conv_block(), параметры которой входные и выходные параметры каналов. Внутри функции последовательные сверточные слои с ядром 3 (3х3) каждый предшествует Relu активационной функции и для лучшей сходимости слои BatchNorm2d:

```
import torch
import torch.nn as nn

# добавляем блок свёрток
def conv_block(in_channels,  out_channels):
    conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3),
        nn.ReLU(),
        nn.BatchNorm2d(num_features=out_channels),
        nn.Conv2d(out_channels, out_channels, kernel_size=3),
        nn.ReLU(),
        nn.BatchNorm2d(num_features=out_channels)
    )
    return conv
```

Шаг 2.

Создадим класс Unet() и сделаем слои левой части и maxpool слои. В каждом слое мы используем conv_block(). Давайте назовем  слои conv_down (4 слоя в левой части): 



```
class Unet(nn.Module):
    def __init__(self, num_classes):
        super(Unet, self).__init__()
        # запоминаем сколько классов
        self.num_classes = num_classes
        self.down_conv_11 = conv_block(in_channels=3, out_channels=64)
        self.down_conv_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_21 = conv_block(in_channels=64, out_channels=128)
        self.down_conv_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_31 = conv_block(in_channels=128, out_channels=256)
        self.down_conv_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_41 = conv_block(in_channels=256, out_channels=512)
        self.down_conv_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = conv_block(in_channels=512, out_channels=1024)
```



Сделаем внутри класса функцию forward(), которой мы отправим входное изображение в левую часть:


```
        def forward(self, X):
        
            x1 = self.down_conv_11(X) # [-1, 64, 256, 256]
            x2 = self.down_conv_12(x1) # [-1, 64, 128, 128]
            x3 = self.down_conv_21(x2) # [-1, 128, 128, 128]
            x4 = self.down_conv_22(x3) # [-1, 128, 64, 64]
            x5 = self.down_conv_31(x4) # [-1, 256, 64, 64]
            x6 = self.down_conv_32(x5) # [-1, 256, 32, 32]
            x7 = self.down_conv_41(x6) # [-1, 512, 32, 32]
            x8 = self.down_conv_42(x7) # [-1, 512, 16, 16]
            middle_out = self.middle(x8) # [-1, 1024, 16, 16]
```



Вот, отлично. Мы создали левую часть нейронной сети. Осталось сделать правую часть.

Шаг 4.

Теперь давайте задекларируем 4 слоя правой части и последнюю 1х1 conv в нашей функции __init__() класса. Вместо maxpool функции мы будем использовать 2х2 transpose convolution, которая будет повышать нашу размерность:

```
        self.up_conv_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512,kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up_conv_12 = conv_block(in_channels=1024, out_channels=512)
        self.up_conv_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up_conv_22 = conv_block(in_channels=512, out_channels=256)
        self.up_conv_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up_conv_32 = conv_block(in_channels=256, out_channels=128)
        self.up_conv_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up_conv_42 = conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
```


Шаг 5.

Как мы видим, в архитектуре входное изображение в правой части - это комбинация изображения с левой части
и с предыдущего слоя. Но для комбинации изображений они должны быть одинаковых размеров. Поэтому давайте создадим функцию crop_tensor() для вырезания этих изображений. Внутри этой функции мы подразумеваем, что наши изображения - это тензоры.

Что происходит в функции crop_tensor() ?

tensor = изображение с левой части, которое необходимо обрезать
target tensor = изображение в правой части, которое сопоставляется с вырезанным левым изображением

Возьмем последний размер обоих тензоров target_size и tensor_size, т.к. их высота и ширина одинаковы. 
Например: x=torch.Size([1,512,64,64]), таким образом x[2] = 64

Теперь мы имея размеры обоих изображений, вычтем размер меньшего тензора из большего. Предположим
target_size = 56 и tensor_size = 64 -> delta(разница между размерами) будет 8.

Но мы ведь будем вырезать изображение из всех углов 'height' * 'width', поэтому мы разделим delta на 2. 
Таким образом, height и width могут быть вырезаны равно:
    8 => h * w=4 * 4

теперь вернем вырезанный тензор
[:,:,] = все измерения
[delta:tensor_size-delta, delta:tensor_size-delta] = вырезанное изображение

[4:64-4, 4:64-4] => 4:60, 4:60 
в примере выше нам необходима картинка 56х56

На картинке ниже показан пример вырезанной высоты:

<img src='https://drive.google.com/uc?export=view&id=1AURG8EdTu1OHHj8nxSRhEsrGqc4WNb5V' width=500>

In [ ]:
def crop_tensor(target_tensor, tensor):
    target_size = target_tensor.size()[2]
    tensor_size = tensor.size()[2]
    delta = tensor_size - target_size
    delta = delta // 2
    
    return tensor[:,:, delta:tensor_size-delta, delta:tensor_size-delta]

Шаг 6.

Теперь допишем наш forward правой части.


Комбинируем оба изображения используя torch.cat() и подставляем в up_conv():



```
        x = self.up_conv_11(middle_out) # [-1, 512, 32, 32]
        y = crop_tensor(x, x7)
        # конкатенируем и сжимаем
        x = self.up_conv_12(torch.cat((x, y), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        
        x = self.up_conv_21(x) # [-1, 256, 64, 64]
        y = crop_tensor(x, x5)
        x = self.up_conv_22(torch.cat((x, y), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        
        x = self.up_conv_31(x) # [-1, 128, 128, 128]
        y = crop_tensor(x, x3)
        x = self.up_conv_32(torch.cat((x, y), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        
        x = self.up_conv_41(x) # [-1, 64, 256, 256]
        y = crop_tensor(x, x1)
        x = self.up_conv_42(torch.cat((x, y), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        
        output = self.output(x) # [-1, num_classes, 256, 256]
        
        return output
```



Теперь для вида запишем наши созданые ранее функции внутрь класса. В итоге наш класс Unet выглядит следующим образом:

In [ ]:
import torch
import torch.nn as nn

class UNet(nn.Module):

    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes

        # Левая сторона (Путь уменьшения размерности картинки)
        self.down_conv_11 = self.conv_block(in_channels=3,
                                            out_channels=64)
        self.down_conv_12 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_21 = self.conv_block(in_channels=64,
                                            out_channels=128)
        self.down_conv_22 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_31 = self.conv_block(in_channels=128,
                                            out_channels=256)
        self.down_conv_32 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_41 = self.conv_block(in_channels=256,
                                            out_channels=512)
        self.down_conv_42 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        
        # Правая сторона (Путь увеличения размерности картинки)
        self.up_conv_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_12 = self.conv_block(in_channels=1024,
                                          out_channels=512)
        self.up_conv_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_22 = self.conv_block(in_channels=512,
                                          out_channels=256)
        self.up_conv_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_32 = self.conv_block(in_channels=256,
                                          out_channels=128)
        self.up_conv_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_42 = self.conv_block(in_channels=128,
                                          out_channels=64)
        
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes,
                                kernel_size=3, stride=1,
                                padding=1)
        self.softmax = nn.Softmax()
    
    @staticmethod
    def conv_block(in_channels, out_channels):
        block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=out_channels),
            nn.Conv2d(in_channels=out_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=out_channels))
        return block
    
    @staticmethod
    def crop_tensor(target_tensor, tensor):
        target_size = target_tensor.size()[2]
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2

        return tensor[:,:, delta:tensor_size-delta, delta:tensor_size-delta]


    def forward(self, X):
        # Проход по левой стороне
        x1 = self.down_conv_11(X) # [-1, 64, 256, 256]
        x2 = self.down_conv_12(x1) # [-1, 64, 128, 128]
        x3 = self.down_conv_21(x2) # [-1, 128, 128, 128]
        x4 = self.down_conv_22(x3) # [-1, 128, 64, 64]
        x5 = self.down_conv_31(x4) # [-1, 256, 64, 64]
        x6 = self.down_conv_32(x5) # [-1, 256, 32, 32]
        x7 = self.down_conv_41(x6) # [-1, 512, 32, 32]
        x8 = self.down_conv_42(x7) # [-1, 512, 16, 16]
        
        middle_out = self.middle(x8) # [-1, 1024, 16, 16]

        # Проход по правой стороне
        x = self.up_conv_11(middle_out) # [-1, 512, 32, 32]
        y = self.crop_tensor(x, x7)
        x = self.up_conv_12(torch.cat((x, y), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        
        x = self.up_conv_21(x) # [-1, 256, 64, 64]
        y = self.crop_tensor(x, x5)
        x = self.up_conv_22(torch.cat((x, y), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        
        x = self.up_conv_31(x) # [-1, 128, 128, 128]
        y = self.crop_tensor(x, x3)
        x = self.up_conv_32(torch.cat((x, y), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        
        x = self.up_conv_41(x) # [-1, 64, 256, 256]
        y = self.crop_tensor(x, x1)
        x = self.up_conv_42(torch.cat((x, y), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        
        output = self.output(x) # [-1, num_classes, 256, 256]
        output = self.softmax(output)

        return output

## Часть 3. Обучение

У нас есть готовые данные и определенная сеть, которую мы хотим обучить. Пришло время построить базовый обучающий конвейер.

Определим скорость обучения и количество эпох:

In [ ]:
learning_rate = 0.001
epochs = 1

Выберем устройство,на котором будем обучать нашу модель:

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

Определим нашу модель Unet для 13 классов:

In [ ]:
Umodel = UNet(num_classes=13).to(device)

In [ ]:
# проверим, что модель не ломается и проходит прямой проход
sample = (next(iter(train_data_loader)))
sample[1].shape

In [ ]:
out = Umodel(sample[0].to(device))
out.shape

In [ ]:
plt.imshow(out[0][2].detach().cpu());

Что-то выходит, не ломается, значит всё ок.

Под обучением мы понимаем скармливание целевой функции оптимизирующей функции. Поэтому выберем оптимизирующую функцию и функцию потерь (целевая функция):

In [ ]:
optimizer = torch.optim.Adam(Umodel.parameters())

In [ ]:
class DiceLoss(nn.Module):
    # функция потерь на основе коэф. Dice
    # в инициализации подтягиваем все методы реализованные в модуле
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    # переопределяем проход на логитах и целевых значениях
    def forward(self, logits, targets):
        # добавляем эпсилон = 1 чтобы не было деления на ноль
        smooth = 1
        # кол-во пришедших объектов в функцию потерь
        num = targets.size(0)
        probs = logits
        m1 = probs.reshape(num, -1)
        m2 = targets.reshape(num, -1)
        intersection = (m1 * m2)
        # коэф Dice чем больше тем лучше
        score = (2. * intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        # т.к. мы привыкли минимизировать функцию потерь, отнормируем сумму на кол-во
        # объектов и вычтем из единицы
        score = 1 - (score.sum() / num)
        return score

Определим количество шагов внутри одной эпохи:

In [ ]:
total_steps = len(train_data_loader)
print(f"{epochs} epochs, {total_steps} total_steps per epoch")

In [ ]:
criterion = DiceLoss()

Запускаем сам процесс обучения:

In [ ]:
#Импортируем библиотеку time для расчета, сколько времени у нас уходит на одну эпоху
import time


# запускаем главный тренировочный цикл
epoch_losses = []
for epoch in range(epochs):
    # запоминаем время начала обучения
    start_time = time.time()
    epoch_loss = []
    
    for batch_idx, (data, labels) in enumerate(train_data_loader):
        
        data, labels = data.to(device), labels.to(device)        
        
        # обнуляем градиенты
        optimizer.zero_grad()
        # прогоняем данные через модель
        outputs = Umodel(data)                
        
        # считаем ошибку
        #loss = nn.CrossEntropyLoss(outputs,labels)# - torch.log(DiceLoss(outputs, labels))
        loss = criterion(outputs, labels)
        
        # подсчёт градиента на обратном проходе
        loss.backward()
        
        # шаг оптимизации, изменяя веса
        optimizer.step()
        
        epoch_loss.append(loss.item())
        
        if batch_idx % 200 == 0:
            print(f'batch index : {batch_idx} | loss : {loss.item()}')

    print(f'Epoch {epoch+1}, loss: ', np.mean(epoch_loss))
    end_time = time.time()
    print(f'Spend time for 1 epoch: {end_time - start_time} sec')
    
    epoch_losses.append(epoch_loss)

Сохраним нашу модель:

In [ ]:
save_model_path = './Unet_Model_dice_loss.pth'

In [ ]:
# сохраняем веса через словарь
torch.save(Umodel.state_dict(), save_model_path)

In [ ]:
# сохраняем архитектуру
net = UNet(13).to(device)
net.load_state_dict(torch.load(save_model_path))

In [ ]:
def get_orig(image):
    image = image.permute(1, 2, 0)
    image = image.numpy()
    image = np.clip(image, 0, 1)
    return image

Проверим работу на тестовых объектах

In [ ]:
class_idx = 1

for i, data in enumerate(test_data_loader):
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    f, axarr = plt.subplots(1,3, figsize=(15, 6))

    for j in range(0, 4):
        axarr[0].imshow(outputs.squeeze().detach().cpu().numpy()[j,class_idx,:,:])
        axarr[0].set_title('Guessed labels')
        axarr[1].imshow(labels.detach().cpu().numpy()[j,class_idx, :,:])
        axarr[1].set_title('Ground truth labels')

        original = get_orig(images[j].cpu())
        axarr[2].imshow(original)
        axarr[2].set_title('Original Images')
        plt.show()
    if i > 3:
        break

## Реализация на PyTorch

Стоит сказать что уже **есть реализация Unet в PyTorch**. Она и другие популярные модели для решения задачи сегментации находятся в библиотеке [segmentation_models_pytorch](https://segmentation-modelspytorch.readthedocs.io/en/latest/index.html)

Если у вас нет этой библиотеки, то для дальнейшей работы вам надо ее установить через pip

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
import segmentation_models_pytorch as smp

# создание модели
BACKBONE = 'resnet34' # сеть для изучения признаков (предобученая)
segmodel = smp.Unet(BACKBONE, classes=13, activation='softmax').to(device) # сеть для сегментации
# препроцессинг берём из imagenet, потому что она была предобучена на imagenet
preprocess_input = smp.encoders.get_preprocessing_fn(BACKBONE, pretrained='imagenet')

In [ ]:
# препроцессим, видим что картинка после препроцессинга стала темнее, но так лучше для сети
dataset = SelfDrivingDataset(df, preprocessing=preprocess_input)
img, masks = dataset[0]
print(img.shape, masks.shape)
fig, ax = plt.subplots(1, 2, figsize=(15, 7))
ax[0].imshow(img.permute(1, 2, 0))
ax[1].imshow(masks.permute(1, 2, 0)[..., 10])
plt.show()

In [ ]:
# 70 % в тренировочную выборку, 30 - в тестовую
X_train, X_test = train_test_split(df, test_size=0.3)

# Упорядочиваем индексацию
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

# Оборачиваем каждую выборку в наш кастомный датасет
train_data = SelfDrivingDataset(X_train,
                                preprocessing=preprocess_input)
test_data = SelfDrivingDataset(X_test,
                               preprocessing=preprocess_input)

In [ ]:
train_data_loader = DataLoader(
    train_data,
    batch_size=8,
    shuffle=True
)
test_data_loader = DataLoader(
    test_data,
    batch_size=4,
    shuffle=False
)

In [ ]:
for img, target in train_data_loader:
    print(img.shape, target.shape)
    print(img[0].min(), img[0].max())
    print(target[0].min(), target[0].max())
    break

In [ ]:
# Dice loss и метрики есть готовые
criterion = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU(),]

optimizer = torch.optim.Adam(params=segmodel.parameters(), lr=0.001)

In [ ]:
# цикл обучения тоже реализован
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    segmodel, 
    loss=criterion, 
    metrics=metrics, 
    optimizer=optimizer,
    device=device,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    segmodel, 
    loss=criterion, 
    metrics=metrics, 
    device=device,
    verbose=True,
)

In [ ]:
# нужно написать только цикл по эпохам
# train model

max_score = 0

for i in range(0, 1):
    print(f'Epoch: {i + 1}')
    train_logs = train_epoch.run(train_data_loader)
    valid_logs = valid_epoch.run(test_data_loader)
    
    # сохраняем модель если метрика лучшая или что-то ещё (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(segmodel, './best_model.pth')
        print('Model saved!')

Видим, что за 1 эпоху переобучения нет. Можно пообучать подольше.

Посмотрим как модель сегментирует.

In [ ]:
class_idx = 1

for i, data in enumerate(test_data_loader):
    images, labels = data
    images = images.to(device)
    labels = labels.to(device)
    outputs = net(images)
    f, axarr = plt.subplots(1,3, figsize=(15, 6))

    for j in range(0, 4):
        axarr[0].imshow(outputs.squeeze().detach().cpu().numpy()[j,class_idx,:,:])
        axarr[0].set_title('Guessed labels')
        axarr[1].imshow(labels.detach().cpu().numpy()[j,class_idx, :,:])
        axarr[1].set_title('Ground truth labels')

        original = get_orig(images[j].cpu())
        axarr[2].imshow(original)
        axarr[2].set_title('Original Images')
        plt.show()
    if i > 3:
        break